In [1]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import nibabel as nib
from skimage.measure import label

In [2]:
def normalize(volume):
    # Normalize into range [0, 1]
    vol_max = volume.max()
    vol_min = volume.min()
    volume = (volume - vol_min) / (vol_max - vol_min)
    return volume


def morphology(seg_vol, kernel_size = (15, 15)):
    (x, y, z) = seg_vol.shape
    kernel1 =  cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    kernel2 =  cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_size)
    for i in range(x):
        axial = cv2.morphologyEx(seg_vol[i,:,:], cv2.MORPH_CLOSE, kernel1)
        seg_vol[i,:,:] = cv2.morphologyEx(axial, cv2.MORPH_OPEN, kernel2)
    """for i in range(y):
        coronal = cv2.morphologyEx(seg_vol[:,i,:], cv2.MORPH_CLOSE, kernel1)
        seg_vol[:,i,:] = cv2.morphologyEx(coronal, cv2.MORPH_OPEN, kernel2)
    for i in range(z):
        sagittal = cv2.morphologyEx(seg_vol[:,:,i], cv2.MORPH_CLOSE, kernel1)
        seg_vol[:,:,i] = cv2.morphologyEx(sagittal, cv2.MORPH_OPEN, kernel2)"""
    return seg_vol
def centroid(img, lcc=False):
    if lcc:
        img = img.astype(np.uint8)
        nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(img, connectivity=4)
        sizes = stats[:, -1]
        if len(sizes) > 2:
            max_label = 1
            max_size = sizes[1]

        for i in range(2, nb_components):
            if sizes[i] > max_size:
                max_label = i
                max_size = sizes[i]

        img2 = np.zeros(output.shape)
        img2[output == max_label] = 255
        img = img2

    if len(img.shape) > 2:
        M = cv2.moments(img[:,:,1])
    else:
        M = cv2.moments(img)

    if M["m00"] == 0:
        return (img.shape[0] // 2, img.shape[1] // 2)
  
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return (cX, cY)

def to_polar(input_img, center):
    #input_img = input_img.astype(np.float32)
    value = np.sqrt(((input_img.shape[0]/2.0)**2.0)+((input_img.shape[1]/2.0)**2.0))
    polar_image = cv2.linearPolar(input_img, center, value, cv2.WARP_FILL_OUTLIERS)
    polar_image = cv2.rotate(polar_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    return polar_image

def to_cart(input_img, center):
    #input_img = input_img.astype(np.float32)
    input_img = cv2.rotate(input_img, cv2.ROTATE_90_CLOCKWISE)
    value = np.sqrt(((input_img.shape[1]/2.0)**2.0)+((input_img.shape[0]/2.0)**2.0))
    polar_image = cv2.linearPolar(input_img, center, value, cv2.WARP_FILL_OUTLIERS + cv2.WARP_INVERSE_MAP)
    #polar_image = polar_image.astype(np.uint8)
    return polar_image

def get_CC_largerThanTh(arr, thresh=8000,dbg=False):
    if dbg:
        dbg_CC(arr, prec=0.02)

    print('Applying Connected Component and take components with num pixels > max_pixels')
    labels = label(arr)
    print('Found ', labels.max(), 'labels')
    max_label = 0
    # Find largestCC
    large_labels = []
    for c_label in range(1, labels.max()+1):
        curr_num_bins = np.sum(np.where(labels == c_label, 1, 0))
        print(c_label, ':', curr_num_bins)
        if curr_num_bins > thresh:
            large_labels.append(c_label)
    print('Max CC label is: ', max_label)

    print('Num liver before CC: ', np.sum(arr))
    is_first = True
    for c_label in large_labels:
        if is_first:
            arr = np.where(labels == c_label, 1, 0)
            is_first = False
        else:
            arr[labels == c_label] = 1
    print('Num liver After CC: ',np.sum(arr) )

    if dbg:
        dbg_CC(arr,prec=0.02)
    return arr

def dbg_CC(arr, prec=0.01):
    from mpl_toolkits.mplot3d import Axes3D
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    pos = np.where(arr == 1)
    num_points = int(np.round(prec * len(pos[0])))
    indices = np.random.permutation(len(pos[0]))[0:num_points]

    ax.scatter(pos[0][indices], pos[1][indices], pos[2][indices])
    ax.view_init(elev=230., azim=360)
    plt.show()
    plt.ioff()
    # plt.waitforbuttonpress()
    plt.close()

def get_crop_coordinates(img, pad_size=2):
    """ input: binaty @D image
        output: crop coordinates of minimal "1" area with gap padding"""
    im_h, im_w = img.shape
    liver_m, liver_n = np.where(img >= 1)
    h_min = min(liver_m) - pad_size
    h_max = max(liver_m) + pad_size
    h = h_max - h_min + 1
    w_min = min(liver_n) - pad_size
    w_max = max(liver_n) + pad_size
    w = w_max - w_min + 1
    gap = abs(h - w)
    pad_l = int(np.ceil(gap / 2.))
    pad_r = int(np.floor(gap / 2.))
    if h > w:
        w_min -= pad_l
        w_max += pad_r
        if w_min < 0:
            w_min = 0
            w_max += (0 - w_min)
        if w_max > im_w:
            w_min -= w_max - im_w
            w_max = im_w
    if h < w:
        h_min -= pad_l
        h_max += pad_r
        if h_min < 0:
            h_min = 0
            h_max += (0 - h_min)
        if h_max > im_h:
            h_min -= h_max - im_h
            h_max = im_h

    return h_min, h_max, w_min, w_max

def get_crop_coordinates_3D(img_arr, pad_size=1,dbg=False):
    """ input: binaty 3D image
        output: global crop coordinates of minimal "1" area with gap padding"""
    im_d, im_h, im_w = img_arr.shape
    liver_z, liver_h, liver_w = np.where(img_arr >= 1)
    z_min = min(liver_z) - pad_size
    z_max = max(liver_z) + pad_size
    z = z_max - z_min + 1    
    h_min = min(liver_h) - pad_size
    h_max = max(liver_h) + pad_size
    h = h_max - h_min + 1
    w_min = min(liver_w) - pad_size
    w_max = max(liver_w) + pad_size
    w = w_max - w_min + 1
    gap = abs(h - w)
    pad_l = int(np.ceil(gap / 2.))
    pad_r = int(np.floor(gap / 2.))
    if h > w:
        w_min -= pad_l
        w_max += pad_r
        if w_min < 0:
            w_min = 0
            w_max += (0 - w_min)
        if w_max > im_w:
            w_min -= w_max - im_w
            w_max = im_w
    if h < w:
        h_min -= pad_l
        h_max += pad_r
        if h_min < 0:
            h_min = 0
            h_max += (0 - h_min)
        if h_max > im_h:
            h_min -= h_max - im_h
            h_max = im_h
    if dbg:
        from mpl_toolkits.mplot3d import Axes3D
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        num_points = int(np.round(0.02 * len(liver_z)))
        indices = np.random.permutation(len(liver_z))[0:num_points]
        # ax.scatter(liver_h, liver_w, liver_z)
        ax.scatter(liver_h[indices], liver_w[indices],liver_z[indices],s=0.8)
        ax.plot([h_min,h_max,h_max,h_min,h_min],[w_min, w_min,w_max, w_max,w_min], zs=int(im_d / 2), zdir='z',color='black')
        ax.view_init(elev=180., azim=360)
        plt.show()
        plt.ioff()
        plt.waitforbuttonpress()
        plt.close()

    return z_min, z_max, h_min, h_max, w_min, w_max

In [3]:
tumor_model = load_model('D:/Study/Thesis/LiTS/models/Best models/Multi-plane/Axial/tumor-liver-crops-512x512-2405_weights_best.h5')
tumor_polar_model = load_model('D:/Study/Thesis/LiTS/models/Best models/Polar/tumor-liver-crops-polart.h5')

In [12]:
test_list = ['106', '107', '108', '109', '110']
slice_list = os.listdir("D:/Study/Thesis/LiTS/datasets-png/test/ct/ct")

In [13]:
z_list = []
for id in test_list:
    count = 0
    for filename in slice_list:
        if id == (filename.split('-')[1]).split('_')[0]:
            count += 1
    z_list.append(count)

In [14]:
z_list

[771, 771, 856, 756, 816]

In [15]:
ct_path = 'D:/Study/Thesis/LiTS/datasets-png/test/ct/ct'
seg_path = "D:/Study/Thesis/LiTS/LiverSeg-UNet-ResNet50-20220528T101045Z-001/LiverSeg-UNet-ResNet50"

In [16]:
save_path = 'D:/Study/Thesis/LiTS/Final-result/Polar'

In [17]:
for i in range(len(test_list)):
    image_arr = np.zeros((z_list[i], 512, 512, 3)).astype('uint8')
    liver_seg = np.zeros((z_list[i], 512, 512)).astype('uint8')
    for j in range(z_list[i]):
        image_arr[j] = cv2.imread(os.path.join(ct_path, f'volume-{test_list[i]}_{j}.png'))
        liver_seg[j] = cv2.imread(os.path.join(seg_path, f'segmentation-{test_list[i]}_{j}.png'), cv2.IMREAD_GRAYSCALE)
    
    liver_seg = liver_seg / 255
    liver_seg[liver_seg >= 0.5] = 1
    liver_seg[liver_seg < 0.5] = 0
    (z, h, w) = liver_seg.shape
    liver_seg = morphology(liver_seg)
    image_arr = normalize(image_arr)
    
    seg_liver_CC = get_CC_largerThanTh(np.where(liver_seg > 0, 1, 0), dbg=False)
    # 3-D Crop coordinates
    (z1, z2, h1, h2, w1, w2) = get_crop_coordinates_3D(seg_liver_CC, dbg=False)
    seg_lesion_arr = np.zeros((z, h, w)).astype('uint8')
    """seg_coronal_arr = np.zeros((z, h, w)).astype('uint8')
    seg_sagittal_arr = np.zeros((z, h, w)).astype('uint8')"""
    # Crop in 3D
    crop_img_arr = image_arr[:, h1:h2, w1:w2, :]
    crop_mask_arr = liver_seg[:, h1:h2, w1:w2]
    liver_crop_h, liver_crop_w = (512, 512)
    _, curr_liver_crop_h, curr_liver_crop_w = crop_mask_arr.shape
    
    for j in range(z):
        if crop_mask_arr[j].sum() == 0:
            pred_tumor = np.zeros((liver_crop_w, liver_crop_h)).astype('uint8')
        else:
            crop_img = cv2.resize(crop_img_arr[j], (liver_crop_w, liver_crop_h), interpolation=cv2.INTER_CUBIC)
            crop_img = np.expand_dims(crop_img, axis=0)
            pred_tumor = tumor_model.predict(crop_img, verbose=1)[0][:,:,0]
            pred_tumor[pred_tumor >= 0.5] = 1
            pred_tumor[pred_tumor < 0.5] = 0
            if pred_tumor.sum() != 0:
                center = centroid(pred_tumor)
                crop_img = to_polar(crop_img[0], center)
                crop_img = np.expand_dims(crop_img, axis=0)
                pred_tumor = tumor_polar_model.predict(crop_img, verbose = 1)[0][:,:,0]
                pred_tumor[pred_tumor >= 0.5] = 1
                pred_tumor[pred_tumor < 0.5] = 0
                pred_tumor = to_cart(pred_tumor, center)

        pred_tumor = cv2.resize(pred_tumor, (curr_liver_crop_h, curr_liver_crop_w), interpolation=cv2.INTER_NEAREST)
        pred_tumor[crop_mask_arr[j] == 0] = 0
        seg_lesion_arr[j, h1:h2, w1:w2] = pred_tumor
    
    liver_seg[seg_lesion_arr == 1] = 2
    liver_seg[liver_seg == 1] = 0
    liver_seg[liver_seg == 2] = 1
    liver_seg = liver_seg * 255
    
    for j in range(z):
        cv2.imwrite(os.path.join(save_path, f'segmentation-{test_list[i]}_{j}.png'), liver_seg[j])

Applying Connected Component and take components with num pixels > max_pixels
Found  7 labels
1 : 3029
2 : 2428
3 : 961
4 : 2368
5 : 2380196
6 : 4808
7 : 3086
Max CC label is:  0
Num liver before CC:  2396876
Num liver After CC:  2380196
1/1 [==============================] - 2s 2s/step
Applying Connected Component and take components with num pixels > max_pixels
Found  12 labels
1 : 1806
2 : 27281
3 : 3023274
4 : 7230
5 : 2634
6 : 5948
7 : 817
8 : 2996
9 : 257
10 : 475
11 : 800
12 : 533
Max CC label is:  0
Num liver before CC:  3074051
Num liver After CC:  3050555
1/1 [==============================] - 2s 2s/step
Applying Connected Component and take components with num pixels > max_pixels
Found  15 labels
1 : 1200149
2 : 415
3 : 11293
4 : 531
5 : 4849872
6 : 64868
7 : 893
8 : 199
9 : 43052
10 : 4308
11 : 15200
12 : 24611
13 : 418
14 : 589
15 : 257
Max CC label is:  0
Num liver before CC:  6216655
Num liver After CC:  6209045
1/1 [==============================] - 2s 2s/step
Applying 

In [18]:
def dice_coef(im1, im2, empty_score=1.0):

    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    im_sum = im1.sum() + im2.sum()
    if im_sum == 0:
        return empty_score

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return (2. * intersection.sum() / im_sum)
def IoU(im1, im2, empty_score = 1.0):
    im1 = np.asarray(im1).astype(bool)
    im2 = np.asarray(im2).astype(bool)
    intersection = np.logical_and(im1, im2)
    union = np.logical_or(im1, im2)
    if np.sum(union) == 0:
        return empty_score 
    return (np.sum(intersection) / np.sum(union))

In [19]:
dice_test = []
iou_test = []
mask_path = 'D:/Study/Thesis/LiTS/datasets-png/test/tumor_seg/tumor_seg'
liver_path = 'D:/Study/Thesis/LiTS/datasets-png/test/liver_seg/liver_seg'
mask_list = os.listdir(save_path)

for filename in mask_list:
    true_mask = cv2.imread(os.path.join(mask_path, filename), cv2.IMREAD_GRAYSCALE)
    pred_mask = cv2.imread(os.path.join(save_path, filename), cv2.IMREAD_GRAYSCALE)
    liver_mask = cv2.imread(os.path.join(liver_path, filename), cv2.IMREAD_GRAYSCALE)
    if liver_mask.sum() != 0:
        dice_test.append(dice_coef(true_mask, pred_mask))
        iou_test.append(IoU(true_mask, pred_mask))


print('Dice Mean:', np.array(dice_test).mean())
print('IoU Mean:', np.array(iou_test).mean())

Dice Mean: 0.7271510670441188
IoU Mean: 0.6835057416261797
